In [1]:
def coupon_csvdata_read(testdata_file_name: str,
                         force_index: int,
                         strain_index: int):
    """
    Read tensile coupon test data from a CSV file and extract force and strain columns.

    Args:
        testdata_file_name (str): Path to the CSV file containing the test data.
        force_index (int): 1-based column index for the force data.
        strain_index (int): 1-based column index for the strain data.

    Returns:
        tuple:
            df_new (pd.DataFrame): DataFrame containing only 'Force' and 'Strain' columns.
            sample_name (str): Name of the sample extracted from the file name (without extension).

    Notes:
        - The CSV file is expected to have a header in the first row.
        - The second row is skipped during reading (`skiprows=[1]`).
        - The first column is treated as the index column (`index_col=0`).
        - Force and strain columns are extracted based on the provided 1-based indices.
    """
    
    import pandas as pd
    df = pd.read_csv(testdata_file_name, skiprows=[1], index_col=0)
    Force = df[df.columns[force_index - 1]]
    Strain = df[df.columns[strain_index - 1]]
    df_new = pd.concat([Force, Strain], axis=1)
    df_new.columns = ['Force', 'Strain']
    sample_name = testdata_file_name[:-4]
    return sample_name, df_new, 

In [2]:
data =coupon_csvdata_read('S5_bench.csv',1,2)

In [3]:
data[0]

'S5_bench'

In [4]:
data[1]

,Force,Strain
Time,,
0.0,0.010520,1.180000e-06
0.5,0.010932,9.160000e-07
1.0,0.010985,1.180000e-06
1.5,0.010842,-1.140000e-06
2.0,0.010288,-3.640000e-06
...,...,...
7394.5,-0.004089,2.241986e-01
7395.0,-0.003981,2.241981e-01
7395.5,-0.004160,2.241971e-01


In [5]:
import pandas as pd
df = pd.read_csv('S5_bench.csv', header=[0])
    
    #df.columns = [f"{col[0]} {col[1]}" for col in df.columns]
    
    # Extract relevant columns
    #time = df["Time (sec)"]
    #displacement = df["Crosshead separation (mm)"]
force = df[df.columns[1]]
elongation = df[df.columns[2]]
strain = elongation # Strain in mm/mm

In [6]:
strain

0        1.180000e-06
1        1.180000e-06
2        9.160000e-07
3        1.180000e-06
4       -1.140000e-06
             ...     
14790    2.241986e-01
14791    2.241981e-01
14792    2.241971e-01
14793    2.241984e-01
14794    2.241979e-01
Name: Extensometer (mm/mm), Length: 14795, dtype: float64

In [46]:
def coupon_test_analysis_base (testdata_file_name: str,
                 thickness: float = 2.5, 
                 width: float = 10, 
                 showfig: bool = True,
                 savefig: bool = False,
                 lower_bound: float = 0.1, 
                 upper_bound: float = 0.3,
                 ):
    """
    Post-process a tensile coupon test and plot stress-strain curve.
     
    To active with Jupyter Lab, '%matplotlib widget' is required
    
    Args:
        Thickness (float): Specimen thickness in mm.
        Width (float): Specimen width in mm.
        file_name (str): CSV file containing test data.
        low_bound (float): Lower bound of elastic region as fraction of UTS.
        upper_bound (float): Upper bound of elastic region as fraction of UTS.

    Returns:
        fig (matplotlib.figure.Figure): Figure object containing the plot.
    """
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    #%matplotlib widget
    # Constants
    thickness # mm
    width # mm
    area = thickness * width  # Calculate the area of the specimen
    
    # Load tensile test data
    df = pd.read_csv(testdata_file_name, header=[0])
    
    #df.columns = [f"{col[0]} {col[1]}" for col in df.columns]
    
    # Extract relevant columns
    #time = df["Time (sec)"]
    #displacement = df["Crosshead separation (mm)"]
    force = df[df.columns[1]]
    elongation = df[df.columns[2]]
    strain = elongation # Strain in mm/mm
    
    # Calculate stress and strain
    force = force * 1000 # Convert kN to N
    stress = (force / area)  # N/m^2 or Pa
    uts = stress.max()
    
    #find the data before uts
    idx_peak = np.argmax(stress)
    strain_up = strain[:idx_peak+1]
    stress_up = stress[:idx_peak+1]
    
    #Boundary for 20% - 50% of UTS
    lower_bound = 0.1 * uts
    upper_bound = 0.3 * uts
    
    elastic_reg = (lower_bound <= stress_up) & (stress_up <= upper_bound)
    
    stress_ela = stress_up[elastic_reg]
    strain_ela = strain_up[elastic_reg] 
    
    E, intercept = np.polyfit(strain_ela, stress_ela, 1)
    #print(f"Young's Modulus is: {E} MPa",)
    E_GPa = E / 1000  # Convert MPa to GPa
    #print(f"Intercept: {intercept} MPa")
   
    # Select over 30% of UTS, as yield stress will over 30% uts
    strain_new = elongation
    stress_new = force / area
    mask = (lower_bound <= stress)
    strain_mask = strain_new[mask]
    stress_mask = stress_new[mask]
    
    offset_decimal = 0.002  # 0.2% in decimal
    offset_line = E * (strain_new - offset_decimal) + intercept

    #Find the Yield strength
    diff = stress_mask - offset_line
    cross_index = np.where(diff <= 0)[0][0] 
    x1 = strain_mask[cross_index-1]
    x2 = strain_mask[cross_index]
    y1 = diff[cross_index-1]
    y2 = diff[cross_index]
    yield_strain = x1 - y1 * (x2 - x1) / (y2 - y1)
    yield_strength = np.interp(yield_strain, strain_new, stress_new)

    #Plot
    fig, ax = plt.subplots(figsize=(10,6))
    
    ax.plot(strain, stress, label='Stress-Strain Curve', color='blue')
    ax.plot(strain_new, offset_line, label='0.2% Offset Strain Line', color='yellow',linestyle = '-.')
    
    ax.axhline(y=yield_strength, label=f'Yield Strength = {yield_strength:.2f} MPa', color='green', linestyle = 'dotted')
    ax.axhline(y=uts, color='red', linestyle = '--', label=f'UTS = {uts:.2f} MPa')
    ax.plot(yield_strain, yield_strength, 'ro', label='Yield Point')
    
    ax.set_xlabel('Strain (mm/mm)')
    ax.set_ylabel('Stress (MPa)')
    ax.set_title(testdata_file_name[:-4]+' Stress-Strain Curve with Mechanical Properties')
    ax.legend()
    ax.grid(True)
    ax.set_ylim(0, 1.2 *uts)

    #Show fig or not
    if showfig == True:
        plt.show()
    else:
        plt.close(fig)

    # Save fig or not
    if savefig == True:
        fig.savefig(testdata_file_name[:-4], dpi=300, bbox_inches='tight')
    
    # Print results
    print(f"Sample: {testdata_file_name[:-4]}")
    print(f"Young's Modulus (E): {E:.2f} MPa")
    print(f"Ultimate Tensile Strength (UTS): {uts:.2f} MPa")
    print(f"Yield Strength: {yield_strength:.2f} MPa")
    print(yield_strain)
    print('-' * 40)
    
    # Prepare results dictionary
    results = {
        "E_MPa": E,
        "UTS_MPa": uts,
        "Yield_Strength_MPa": yield_strength
    }
    return cross_index, force[10], x1

In [47]:
%matplotlib widget
import numpy as np
import pandas as pd
#np.set_printoptions(threshold=1000) 
#pd.set_option('display.max_rows', None)  
coupon_test_analysis_base('S5_bench.csv',showfig=0)

Sample: S5_bench
Young's Modulus (E): 231643.72 MPa
Ultimate Tensile Strength (UTS): 402.11 MPa
Yield Strength: 331.04 MPa
0.0031348473971851377
----------------------------------------


(np.int64(845), np.float64(10.627659), np.float64(0.0031256))